## Codice eseguito su Google Colab Pro per sfruttare anche il calcolo su GPU che velocizza nettamente il train dei modelli

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pytorch-tabular

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.1/778.1 kB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 84.5 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=dc9bba995a0fac3154f28ed0fa4c2041241cbf46a110289944f754c414563e75
  Stored in directory: /root/.cache/pip/whee

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
from pytorch_tabular import TabularModel
from pytorch_tabular.models import TabTransformerConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig


import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Progetto Data Analytics/train.csv')
df = df.drop_duplicates()
print(df.shape)
df.head()

(252123, 91)


,Year,S0,S1,S2,S3,S4,S5,S6,S7,S8,...,S80,S81,S82,S83,S84,S85,S86,S87,S88,S89
0,2007,44.76752,114.82099,3.83239,27.99928,1.49153,-15.90853,28.24844,3.61650,-7.24653,...,-1.89619,-471.02844,411.56205,443.01198,19.30254,309.07806,-336.91706,-14.70547,-474.44157,31.32820
1,2004,52.28942,75.73319,11.35941,-6.20582,-27.64559,-30.75995,12.50955,7.47877,9.88498,...,4.57060,1.36110,-6.52977,59.48672,3.69790,-36.92252,44.08077,3.39993,-70.07591,3.86143
2,2005,33.81773,-139.07371,134.19332,17.85216,63.47408,-25.28005,-34.65911,-5.99135,1.27848,...,54.16608,15.04530,39.09107,39.03041,3.68708,-61.88547,45.68115,6.39822,3.24471,35.74749
3,1998,41.60866,3.17811,-3.97174,23.53564,-19.68553,20.74407,18.80866,6.24474,-7.98424,...,28.08591,295.88684,54.02395,102.02880,40.47711,15.10258,-250.32293,2.81288,56.05172,3.60432
4,1987,44.49525,-32.25270,58.08217,3.73684,-32.53274,-18.72885,-15.85665,-3.34607,22.63786,...,31.44988,-136.50457,-85.11989,-74.96342,9.56921,-100.61689,-133.29315,9.19246,-97.37953,30.11015


### Split del dataset

In [5]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=1)
train_df, valid_df = train_test_split(train_df, test_size=0.1, random_state=1)

### pytorch_tabular richiede il nome della colonna target e una lista con tutti i nomi delle colonne continue, oltre che quelle categoriche (non presenti nel nostro dataset). Di seguito la creazione della lista

In [6]:
tutte_le_colonne = df.columns.tolist()
colonna_target = 'Year'
colonne_continue = [col for col in tutte_le_colonne if col != colonna_target]

In [ ]:
from sklearn.metrics import mean_squared_error

batch_sizes = [64, 128, 256, 512]

def train_evaluate_model(batch_size):

    data_config = DataConfig(
        target=['Year'],
        continuous_cols=colonne_continue,
        categorical_cols=[],
        # normalize_continuous_features=True
    )

    model_config = TabTransformerConfig(
        task="regression"
    )

    optimizer_config = OptimizerConfig()

    trainer_config = TrainerConfig(
        max_epochs=200,
        batch_size=batch_size,
        auto_select_gpus=True,
        early_stopping_patience=10
    )

    model = TabularModel(
        data_config=data_config,
        model_config=model_config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config
    )

    model.fit(train=train_df, validation=valid_df)

    predictions = model.predict(valid_df)
    mse = mean_squared_error(valid_df['Year'], predictions)

    return mse

best_batch_size = None
best_mse = float('inf')

for batch_size in batch_sizes:
    mse = train_evaluate_model(batch_size)
    if mse < best_mse:
        best_mse = mse
        best_batch_size = batch_size

print(f"Miglior batch_size: {best_batch_size} con MSE: {best_mse}")

Epoch 42/199 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355/355 0:00:11 • 0:00:00 31.61it/s v_num: 10.000 train_loss: 102.589
                                                                                  valid_loss: 84.817               
                                                                                  valid_mean_squared_error: 84.817 
                                                                                  train_mean_squared_error: 84.487 

INFO:pytorch_tabular.tabular_model:Training the model completed
INFO:pytorch_tabular.tabular_model:Loading the best model


Miglior batch_size: 512 con MSE: 84.75704751705689


### Selezione degli iperparametri (manuale). Aumentando num_heads, num_attn_blocks e ff_hidden_multiplier i risultati non migliorano, restano invariati. Solo l'inserimento della normalizzazione batch sulle feature continue migliora leggermente le prestazioni.

In [7]:
data_config = DataConfig(
        target=['Year'],
        continuous_cols=colonne_continue,
        categorical_cols=[],
        continuous_feature_transform='quantile_uniform'
    )

model_config = TabTransformerConfig(
    task="regression",
    #num_heads=12,
    #num_attn_blocks=8,
    #ff_hidden_multiplier=8,
    batch_norm_continuous_input=True,
    seed=42
)

optimizer_config = OptimizerConfig()

trainer_config = TrainerConfig(
    max_epochs=200,
    batch_size=256,
    auto_select_gpus=True,
    early_stopping_patience=10
)

model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config
)


model.fit(train=train_df, validation=valid_df)

INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off
INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for regression task
INFO:pytorch_tabular.tabular_model:Preparing the Model: TabTransformerModel
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_tabular.tabular_model:Training Started
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                   ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ TabTransformerBackbone │  271 K │
│ 1 │ _embedding_layer │ Embedding2dLayer       │      0 │
│ 2 │ _head            │ LinearHead             │     91 │
│ 3 │ loss             │ MSELoss                │      0 │
└───┴──────────────────┴────────────────────────┴────────┘

Trainable params: 271 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 271 K                                                                                                
Total estimated model params size (MB): 1

Output()

INFO:pytorch_tabular.tabular_model:Training the model completed
INFO:pytorch_tabular.tabular_model:Loading the best model


### Valutazione sul dataset di test

In [11]:
valutazione = model.evaluate(test_df)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │     81.7916259765625      │
│  test_mean_squared_error  │     81.7916259765625      │
└───────────────────────────┴───────────────────────────┘

### Calcolo delle metriche sul dataset di test

In [12]:
y_pred = model.predict(test_df)
y_true = test_df['Year']
print("MSE:", mean_squared_error(y_true, y_pred))
print("RMSE:", np.sqrt(mean_squared_error(y_true, y_pred)))
print("MAE:", mean_absolute_error(y_true, y_pred))
print("MAPE:", mean_absolute_percentage_error(y_true, y_pred))
print("R2:", r2_score(y_true, y_pred))

MSE: 81.7916385481626
RMSE: 9.043872983858332
MAE: 6.515582824313647
MAPE: 0.0032704864647539657
R2: 0.25406473763916704


In [13]:
import torch
torch.save(model, "modelli/TABTRANSFORMER/model_tab_transformer.save")